In [ ]:
# Import the required modules
import pandas as pd
from pathlib import Path
import hvplot.pandas

## Load the Credit Card Data into a Pandas DataFrame

In [ ]:
# Read in the CSV file as a Pandas Dataframe
ccinfo_df = pd.read_csv(
    Path("../Resources/cc_info_default.csv")
)

In [ ]:
# Review the DataFrame
ccinfo_df.tail()

In [ ]:
# Review the info
ccinfo_df.info()

## Transform "education" column with get_dummies

In [ ]:
# Verify the categories of the "education" column
ccinfo_df["education"].value_counts()

In [ ]:
# Transform the education column using get_dummies
education_dummies = pd.get_dummies(ccinfo_df["education"])

# Display the transformed data
education_dummies.tail()

In [ ]:
# Concatenate the df_shopping_transformed and the card_dummies DataFrames
ccinfo_df = pd.concat([ccinfo_df, education_dummies], axis=1)

# Drop the original education column
ccinfo_df = ccinfo_df.drop(columns=["education"])

# Display the DataFrame
ccinfo_df.head()

## Transform "marriage" column with encoding function

In [ ]:
# Encoding the marriage column using a custom function
def encode_marriage(marriage):
    """
    This function encodes marital status by setting yes as 1 and no as 0.
    """
    if marriage == "yes":
        return 1
    else:
        return 0

# Call the encode_marriage function on the marriage column
ccinfo_df["marriage"] = ccinfo_df["marriage"].apply(encode_marriage)

# Review the DataFrame 
ccinfo_df.head()

## Apply the Standard Scaler to "limit_bal", "bill_amt", "pay_amt"

In [ ]:
# Import the module
from sklearn.preprocessing import StandardScaler

In [ ]:
# Scaling the numeric columns
ccinfo_data_scaled = StandardScaler().fit_transform(ccinfo_df[["limit_bal", "bill_amt", "pay_amt"]])

# Review the scaled data
ccinfo_data_scaled

In [ ]:
# Create a DataFrame of the scaled data
ccinfo_data_scaled = pd.DataFrame(ccinfo_data_scaled, columns=["limit_bal", "bill_amt", "pay_amt"])

# Replace the original data with the columns of information from the scaled Data
ccinfo_df["limit_bal"] = ccinfo_data_scaled["limit_bal"]
ccinfo_df["bill_amt"] = ccinfo_data_scaled["bill_amt"]
ccinfo_df["pay_amt"] = ccinfo_data_scaled["pay_amt"]

# Review the DataFrame
ccinfo_df.head()

## Elbow Method to find k

In [ ]:
# Import the KMeans module from SKLearn
from sklearn.cluster import KMeans

In [ ]:
# Create a a list to store inertia values and the values of k
inertia = []
k = list(range(1, 11))

In [ ]:
# Create a for-loop where each value of k is evaluated using the K-means algorithm
# Fit the model using the service_ratings DataFrame
# Append the value of the computed inertia from the `inertia_` attribute of the KMeans model instance
for i in k:
    k_model = KMeans(n_clusters=i, random_state=0)
    k_model.fit(ccinfo_df)
    inertia.append(k_model.inertia_)
    

In [ ]:
# Define a DataFrame to hold the values for k and the corresponding inertia
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

# Review the DataFrame
df_elbow.head()

In [ ]:
# Plot the DataFrame
df_elbow.hvplot.line(
    x="k", 
    y="inertia", 
    title="Elbow Curve", 
    xticks=k
)

## Kmeans algo to cluster data

In [ ]:
# Define the model with 3 clusters
model = KMeans(n_clusters=3, random_state=3)

# Fit the model
model.fit(ccinfo_df)

# Make predictions
k_3 = model.predict(ccinfo_df)

# Create a copy of the preprocessed data
ccinfo_predictions_df = ccinfo_df.copy()

# Add a class column with the labels
ccinfo_predictions_df['customer_segments'] = k_3

In [ ]:
# Plot the clusters
ccinfo_predictions_df.hvplot.scatter(
    x="limit_bal",
    y="age",
    by="customer_segments"
)